## USE THIS DATASET INSTEAD? http://www.fao.org/faostat/en/#data/QC

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
data = pd.read_csv('raw_data/fao_data_crops_data.csv.zip', compression='zip', header=0, sep=',', quotechar='"')
data.head(5)

,country_or_area,element_code,element,year,unit,value,value_footnotes,category
0,Americas +,31,Area Harvested,2007.0,Ha,49404.0,A,agave_fibres_nes
1,Americas +,31,Area Harvested,2006.0,Ha,49404.0,A,agave_fibres_nes
2,Americas +,31,Area Harvested,2005.0,Ha,49404.0,A,agave_fibres_nes
3,Americas +,31,Area Harvested,2004.0,Ha,49113.0,A,agave_fibres_nes
4,Americas +,31,Area Harvested,2003.0,Ha,48559.0,A,agave_fibres_nes


## Explanation of crops data file

Each row of the crops dataset contains data for a certain country/area and year.  
There are 8 columns of metadata, which can be seen below.
The years span from 1961 to 2007, but some years are undefined.  

| Column name         | Explanation          |
|---------------------|----------------------|
| country_or_area     | Name of country/area |
| year                | Unique code for each country/area |
| element             | Data classification type |
| element_code        | Unique code for each type of Element |
| unit                | Unit of measurement |
| value               | The value of the measurement |
| value_footnote      | Where the data comes from |
| category            | Crop category |

#### Footnotes
The value footnotes used in the dataset have the following explanations:

|  Footnote  | Meaning of footnote    |
|------------|------------------------|
| Fc         | Calculated data        |
| A          | Aggregate, may include official, semi-official or estimated or calculated data |
| NR         | Not reported by country|
| F          | FAO Estimate           |

#### Elements
- **Area harvested** refer to the area under cultivation. Area under cultivation means the area that corresponds to the total sown area, but after the harvest it excludes ruined areas (e.g. due to natural disasters). If the same land parcel is used twice in the same year, the area of this parcel can be counted twice. 
- **Production quantity** means the harvested production. Harvested production means production including on-holding losses and wastage, quantities consumed directly on the farm and marketed quantities, indicated in units of basic product weight. *Harvest year* means the calendar year in which the harvest begins. 
- **Yield** means the harvested production per ha for the area under cultivation. 
- **Seed** quantity comprises all amounts of the commodity in question used during the reference period for reproductive purposes, such as seed or seedlings. Usually, the average seed rate in any given country does not vary greatly from year to year.

#### Categories
Crop statistics are recorded for 173 products, covering the following categories: Crops Primary, Fibre Crops Primary, Cereals, Coarse Grain, Citrus Fruit, Fruit, Jute Jute-like Fibres, Oilcakes Equivalent, Oil crops Primary, Pulses, Roots and Tubers, Treenuts and Vegetables and Melons. The objective is to comprehensively cover production of all primary crops for all countries and regions in the world. 

**Cereals**: Area and production data on cereals relate to crops harvested for dry grain only. Cereal crops harvested for hay or harvested green for food, feed or silage or used for grazing are therefore excluded. 

**Vegetables**, total (including melons): Data relate to vegetable crops grown mainly for human consumption. Crops such as cabbages, pumpkins and carrots, when explicitly cultivated for animal feed, are therefore excluded. Statistics on vegetables are not available in many countries, and the coverage of the reported data differs from country to country. In general, it appears that the data refer to crops grown in field and market gardens mainly for sale, thus excluding crops cultivated in kitchen gardens or small family gardens mainly for household consumption.

**Fruit**, total (excluding melons): Data refer to total production of fresh fruit, whether finally used for direct consumption for food or feed, or processed into different products: dry fruit, juice, jam, alcohol, etc. Generally, production data relate to plantation crops or orchard crops grown mainly for sale. Data on production from scattered trees used mainly for home consumption are not usually collected. Production from wild plants, particularly berries, which is of some importance in certain countries, is generally disregarded by national statistical services. Therefore, the data for the various fruits and berries are rather incomplete. Dates, plantains and total grapes are included in the “total fruit” aggregated figures, while olives are excluded.

**Bananas and plantains**: Figures on bananas refer, as far as possible, to all edible fruit-bearing species of the genus Musa except Musa paradisiaca, commonly known as plantain. Unfortunately, several countries make no distinction in their statistics between bananas and plantains and publish only overall estimates. When this occurs and there is some indication or assumption that the data reported refer mainly to bananas, the data are included. The production data on bananas and plantains reported by the various countries are also difficult to compare because a number of countries report in terms of bunches, which generally means that the stalk is included in the weight.  

**Treenuts**, aggregated: Production of nuts (including chestnuts) relates to nuts in the shell or in the husk. Statistics are very scanty and generally refer only to crops for sale. In addition to the kind of nuts shown separately, production data include all other treenuts mainly used as dessert or table nuts, such as pecan nuts, pili nuts, sapucaia nuts and macadamia nuts. Nuts mainly used for flavouring beverages are excludedas are masticatory and stimulant nuts and nuts used mainly for the extraction of oil or butter, including areca/betel nuts, cola nuts, illipe nuts, karate nuts, coconuts, tung nuts, oilpalm nuts etc.

## Cleaning the data

**Renaming**  
First of all we rename the columns to fit the other datasets.

In [27]:
# Rename Columns to match other dataset
new_names = {'country_or_area': 'Area', 'element_code': 'Element Code', 
             'element': 'Element', 'year': 'Year', 'unit': 'Unit', 
             'value': 'Value', 'value_footnotes': 'Flag', 'category': 'Item'}

data.rename(columns = new_names, inplace = True)


#### Unnecessary rows:
The dataset is ordered according to the alphabetical order of the categories and after the end of each Item, there are rows that contain the footnote-descriptions above. We therefore need to remove these rows since they don't contain any useful data.

In [28]:
# Removing rows that do not contain useful information
keep_col = np.logical_not(data.Area.isin(['fnSeqID', 'Fc', 'A ', 'NR', 'F ', '* ']))
crops_data = data[keep_col]

#### Missing information

Are we now missing any information in our datasets?

In [29]:
print("Missing information in country based dataset: \n", crops_data.isna().sum())

Missing information in country based dataset: 
 Area                 0
Element Code         0
Element              0
Year                 0
Unit                 0
Value                0
Flag            478418
Item                 0
dtype: int64


We can see that a lot of value footnotes have missing information. Can we find something in common for these missing values?

In [30]:
missing_values = crops_data[crops_data.Flag.isna()]
print("Number of unique countries included in missing data: ", missing_values.Area.unique().shape[0])
print("Number of unique elements included in missing data: ", missing_values.Element.unique().shape[0])
print("Number of unique years included in missing data: ", missing_values.Year.unique().shape[0])
print("Number of unique categories included in missing data: ", missing_values.Item.unique().shape[0])
print("Number of unique units included in missing data: ", missing_values.Unit.unique().shape[0])

Number of unique countries included in missing data:  217
Number of unique elements included in missing data:  4
Number of unique years included in missing data:  47
Number of unique categories included in missing data:  158
Number of unique units included in missing data:  3


We aren't able to detect if the missing data follows a general rule. The missing data seems to occur in many different cases.

#### Elements

In [31]:
# Aggregate year column to 'min - max' year
def agg_year(series):
    min_year = int(series.min())
    max_year = int(series.max())
    return '{} to {}'.format(min_year, max_year)

# Examine the different countries
def count_unique_area(series):
    return len(series.unique())

# Group by element code and element to see what these columns represent 
crops_continent.groupby(['Element Code', 'Element'])\
             .agg({'value':'sum', 'Unit':'unique', 'Year':agg_year, 'continent':count_unique_area})\
             .sort_values(by='Value', ascending=False)

KeyError: 'Element Code'

By taking the sum of all values and sorting we see that 51-Production quantity has the greatest value. We can observe that, depending one the element, we have different number of unique countries in that column

**Observation:** Are the elements apart form Production Quantity, Area Harvested, Seed and Yield really necessary?

#### Element codes > 140

In [32]:
elem_codes = ['152', '154', '434', '438', '432', '436']
study_data = crops_data[crops_data['Element Code'].isin(elem_codes)]
print("Number of countries in this data: ", study_data.Area.unique().shape[0])
study_data

Number of countries in this data:  222


,Area,Element Code,Element,Year,Unit,Value,Flag,Item
364540,Afghanistan,152,Gross Production 1999-2001 (1000 I$),2007.0,1000 Int. $,896908.0,A,cereals_total
364541,Afghanistan,152,Gross Production 1999-2001 (1000 I$),2006.0,1000 Int. $,718798.0,A,cereals_total
364542,Afghanistan,152,Gross Production 1999-2001 (1000 I$),2005.0,1000 Int. $,841895.0,A,cereals_total
364543,Afghanistan,152,Gross Production 1999-2001 (1000 I$),2004.0,1000 Int. $,549037.0,A,cereals_total
364544,Afghanistan,152,Gross Production 1999-2001 (1000 I$),2003.0,1000 Int. $,685931.0,A,cereals_total
...,...,...,...,...,...,...,...,...
457415,Zimbabwe,438,Net per capita PIN (base 1999-2001),1965.0,Int. $,155.0,A,cereals_total
457416,Zimbabwe,438,Net per capita PIN (base 1999-2001),1964.0,Int. $,146.0,A,cereals_total
457417,Zimbabwe,438,Net per capita PIN (base 1999-2001),1963.0,Int. $,151.0,A,cereals_total
457418,Zimbabwe,438,Net per capita PIN (base 1999-2001),1962.0,Int. $,184.0,A,cereals_total


From what we can see in the new dataset that only contain these elements, both the rows in the beginning and end have the Item cereals_total. Is this the only Item?

In [33]:
study_data.Item.unique()

array(['cereals_total'], dtype=object)

Apparently so. Does this catgory exist for the other type of elements as well? Because if it does, we should be able to remove the elements with codes 152-438 from our dataset.

In [34]:
study_data_2 = crops_data[crops_data.Item.str.contains('cereals_total')]
study_data_2 = study_data_2[np.logical_not(study_data_2['Element Code'].isin(elem_codes))]
print("Number of countries in this data: ", study_data_2.Area.unique().shape[0])
study_data_2['Element Code'].unique()

Number of countries in this data:  225


array(['111', '31', '41', '51'], dtype=object)

We can now see that the data for the Item 'cereals_total' with element codes > 140 are subgroups of element codes < 140. We can therefore remove this data from our dataset.

In [35]:
# Drop rows with element codes > 140
drop_col = np.logical_not(crops_data['Element Code'].isin(elem_codes))
crops_data = crops_data[drop_col]
crops_data

,Area,Element Code,Element,Year,Unit,Value,Flag,Item
0,Americas +,31,Area Harvested,2007.0,Ha,49404.0,A,agave_fibres_nes
1,Americas +,31,Area Harvested,2006.0,Ha,49404.0,A,agave_fibres_nes
2,Americas +,31,Area Harvested,2005.0,Ha,49404.0,A,agave_fibres_nes
3,Americas +,31,Area Harvested,2004.0,Ha,49113.0,A,agave_fibres_nes
4,Americas +,31,Area Harvested,2003.0,Ha,48559.0,A,agave_fibres_nes
...,...,...,...,...,...,...,...,...
2255338,World +,51,Production Quantity,1965.0,tonnes,150123.0,A,yautia_cocoyam
2255339,World +,51,Production Quantity,1964.0,tonnes,143203.0,A,yautia_cocoyam
2255340,World +,51,Production Quantity,1963.0,tonnes,142094.0,A,yautia_cocoyam
2255341,World +,51,Production Quantity,1962.0,tonnes,123840.0,A,yautia_cocoyam


#### Names with '+'

There are countries/areas that contain a '+' at the end of the name. What names contain this sign and what do they have in common?

In [36]:
# Examining what names contain '?'
country_series = crops_data.Area
names_with_sign = country_series[country_series.str.endswith('+')]
names_with_sign.unique()

array(['Americas +', 'Asia +', 'Caribbean +', 'Central America +',
       'Low Income Food Deficit Countries +',
       'Net Food Importing Developing Countries +',
       'Small Island Developing States +', 'South America +',
       'South-Eastern Asia +', 'World +', 'Africa +',
       'Australia and New Zealand +', 'Central Asia +', 'Eastern Asia +',
       'Eastern Europe +', 'Europe +', 'European Union +',
       'LandLocked developing countries +', 'Least Developed Countries +',
       'Northern Africa +', 'Northern America +', 'Oceania +',
       'Southern Africa +', 'Southern Asia +', 'Southern Europe +',
       'Western Africa +', 'Western Asia +', 'Western Europe +',
       'Eastern Africa +', 'Northern Europe +', 'Middle Africa +',
       'Micronesia +', 'Polynesia +', 'Melanesia +'], dtype=object)

All of the country/area names that contain a '+' at the end are all areas. We can therefore divide the dataset into several groups: one with all countries, one with all continents and one with the remaining areas.

## Splitting the dataset 

In [38]:
# Splitting crops_data into country, continent and area based sets and renaming Area column

crops_country = crops_data[np.logical_not(country_series.str.endswith('+'))].rename({'Area': 'Area'}, axis=1)
crops_remain = crops_data[country_series.str.endswith('+')]

# Remove last two characters from continent/area name
crops_remain.Area = crops_remain.Area.str[:-2]

continents = ['Africa', 'Northern America', 'South America', 'Asia', 'Oceania', 'Europe']
is_continent = crops_remain.Area.isin(continents)

crops_continent = crops_remain[is_continent].rename({'Area': 'Area'}, axis=1)
crops_area = crops_remain[np.logical_not(is_continent)].rename({'Area': 'Area'}, axis=1)

print('Number of unique countries:', crops_country.Area.unique().shape[0])
print('Number of unique continents:', crops_continent.Area.unique().shape[0])
print('Number of unique areas:', crops_area.Area.unique().shape[0])

Number of unique countries: 219
Number of unique continents: 6
Number of unique areas: 28


In [39]:
crops_country

,Area,Element Code,Element,Year,Unit,Value,Flag,Item
564,Colombia,31,Area Harvested,2007.0,Ha,17500.0,F,agave_fibres_nes
565,Colombia,31,Area Harvested,2006.0,Ha,17500.0,F,agave_fibres_nes
566,Colombia,31,Area Harvested,2005.0,Ha,17500.0,F,agave_fibres_nes
567,Colombia,31,Area Harvested,2004.0,Ha,17294.0,NaN,agave_fibres_nes
568,Colombia,31,Area Harvested,2003.0,Ha,17094.0,NaN,agave_fibres_nes
...,...,...,...,...,...,...,...,...
2255150,"Venezuela, Bolivarian Republic of",51,Production Quantity,1965.0,tonnes,61062.0,NaN,yautia_cocoyam
2255151,"Venezuela, Bolivarian Republic of",51,Production Quantity,1964.0,tonnes,59225.0,NaN,yautia_cocoyam
2255152,"Venezuela, Bolivarian Republic of",51,Production Quantity,1963.0,tonnes,57500.0,NaN,yautia_cocoyam
2255153,"Venezuela, Bolivarian Republic of",51,Production Quantity,1962.0,tonnes,55825.0,NaN,yautia_cocoyam


In [40]:
# Save dataframes to CSV
#crops_country.to_csv('./data/csv/crops_countries.csv')
#crops_area.to_csv('./data/csv/crops_areas.csv')
#crops_continent.to_csv('./data/csv/crops_continents.csv')

In [41]:
# Save dataframes to pickles
crops_country.to_pickle('./data/pickles/crops_countries.pkl')
crops_area.to_pickle('./data/pickles/crops_areas.pkl')
crops_continent.to_pickle('./data/pickles/crops_continents.pkl')